In [1]:
import numpy as np
import pandas as pd
import torch

import torch.nn as nn
from torch.utils.data import DataLoader

from Dataset.emb_auto import AutoDataset
from Model.AE import Auto

import utils

import matplotlib.pyplot as plt

In [2]:
# 데이터 다운 & 전처리

df = pd.read_feather('../데이터/기본데이터.feather')

try: del df['Unnamed: 0']
except: pass

In [14]:
# df[0] = df['0'].apply(utils.to_list)
# df[1] = df['1'].apply(utils.to_list)

In [18]:
df[0] = df['0']
df[1] = df['1']

In [19]:
len_df = len(df)
train_len = int(len_df *0.6)
val_len = int(len_df * 0.3)

batch_size = 64

train_dataset = AutoDataset(df[:train_len])
train_loader = DataLoader(train_dataset, batch_size=batch_size)

val_dataset = AutoDataset(df[train_len:train_len+val_len])
val_loader = DataLoader(val_dataset, batch_size=batch_size)

test_dataset = AutoDataset(df[train_len+val_len:])
test_loader = DataLoader(test_dataset, batch_size=batch_size)

c:\Users\USER\Desktop\sci\SCI\코드\Dataset\emb_auto.py:16: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xe . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem . (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  self.x = torch.FloatTensor(X)


In [20]:
for x,y in train_loader:
  print("X 크기 : {}".format(x.shape))
  print("Y 크기 : {}".format(y.shape))
  break

X 크기 : torch.Size([64, 7])
Y 크기 : torch.Size([64, 7])


In [21]:
input_size = 7
Model = Auto(input_size)

In [22]:
# 데이터 & 모델에 device 붙임!!!
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

model = Model.to(device)

print('작동하는지 실험')
basic_data = torch.rand((1,input_size))
model(basic_data)


cpu is available
작동하는지 실험


(tensor([[0.0000, 0.0000, 0.0139, 0.1232, 0.0735, 0.0148, 0.0000, 0.0000, 0.0000,
          0.0792, 0.0000, 0.0000, 0.0000, 0.0000, 0.0888, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0149, 0.0047, 0.0754, 0.0000, 0.0100, 0.0000,
          0.0403, 0.1067, 0.0460, 0.0000, 0.1756, 0.0027, 0.0000, 0.1208, 0.0000,
          0.0000, 0.0273, 0.0000, 0.1229, 0.0759, 0.1164, 0.0000, 0.1078, 0.0137,
          0.0797, 0.0000, 0.0133, 0.0000, 0.0250, 0.0000, 0.0000, 0.0017, 0.1379,
          0.0570, 0.0000, 0.0685, 0.0380, 0.0480, 0.1049, 0.0077, 0.0000, 0.0000,
          0.0000, 0.0124, 0.0000, 0.0000, 0.0000, 0.0000, 0.1355, 0.0000, 0.0228,
          0.0149, 0.0542, 0.1049, 0.0000, 0.0883, 0.0000, 0.0476, 0.0000, 0.2082,
          0.0043, 0.0000, 0.0000, 0.0951, 0.1261, 0.0782, 0.0000, 0.0000, 0.1348,
          0.2457, 0.0000, 0.0000, 0.0000, 0.0000, 0.1671, 0.0000, 0.1233, 0.1318,
          0.0081, 0.0000, 0.1436, 0.0000, 0.1254, 0.1045, 0.0000, 0.0000, 0.0131,
          0.0000

In [23]:
criterion = nn.MSELoss()

lr = 1e-3
num_epochs = 1
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [24]:
## 학습하는 코드를 짜봐!!!

train_n = len(train_loader)
val_n = len(val_loader)

# print(train_loader)
best_val_loss = np.inf

for epoch in range(num_epochs):
    print('epoch : {}'.format(epoch))
    running_loss = 0.0
    num = 0
    print("train")
    model.train()
    for inputs, labels in train_loader:  # dataloader 함수에 def __len__에 return 값만큼 반복함
        inputs = inputs.to(device)
        labels = labels.to(device)
        vec, out = model(inputs)
        loss = criterion(out, labels)

        optimizer.zero_grad() #
        loss.backward() # loss가 최소가 되게하는
        optimizer.step() # 어떤 방법으로 learning rate를 움직일거이냐
        running_loss += loss.item() # 한 배치의 loss 더해주고,


    print(running_loss/train_n)

    with torch.no_grad():
            print("Calculating validation results...")
            model.eval()
            val_loss = 0
            for inputs, labels in val_loader:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    vec, outs = model(inputs)

                    loss= criterion(outs, labels)
                    val_loss += loss.item()

            avg_val_loss = val_loss/val_n
            if avg_val_loss < best_val_loss:
                    best_val_loss = avg_val_loss
                    stop_counter = 0            
                    torch.save(model.state_dict(), 'Checkpoint/embedding_best_model.pth')


            # if val_acc > best_val_acc:
            #     print(f"New best model for val accuracy : {val_acc:4.2%}! saving the best model..")
            #     # torch.save(model.module.state_dict(), f"{save_dir}/best.pth")
            #     best_val_acc = val_acc
            # torch.save(model.module.state_dict(), f"{save_dir}/last.pth")
            print("val")
            print(
                f"loss: {val_loss/val_n:4.8} || "
            )
    print('======================================================')



epoch : 0
train


### 더미데이터 생성

In [84]:
data = []
model = model.requires_grad_(False)
for i, (x,_) in enumerate(test_dataset):
    data.append(str(model(x)[0].tolist()))
    if i==100:
        break

In [87]:
pd.DataFrame(data).to_excel('../데이터/인베딩더미.xlsx')